In [1]:
from time import time
import os, sys, csv
import numpy as np
from pylab import *
import pylab as pylab
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from MulticoreTSNE import MulticoreTSNE as TSNE
from scipy import stats


from kpal.klib import Profile

import pandas as pd

import umap

from sklearn import manifold
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn import svm
import umap
from sklearn.datasets import load_digits
from Bio import SeqIO
from io import StringIO
import sys

/home/jacob/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def calc_kmer_freqs(split_seqs, kmer_size):
    '''
    Use kpal to calculate kmer frequencies for split sequences
    '''

    kmer_freqs = []
    for seq in split_seqs:
        temp_list = []

        #for some reason this kmer counter function only works on iterable(str) type objects.
        temp_list.append(str(seq))
        ktable = Profile.from_sequences(temp_list, kmer_size, name=None)

        #skip sequences with a lot of Ns/characters besides A|T|C|G
        if len(str(seq)) < 3000:
            if ktable.total >= len(str(seq))/2:
                ktable.counts = [count/ktable.total for count in ktable.counts]
                kmer_freqs.append(ktable.counts)
        else:
            if ktable.total >= 1500:
                ktable.counts = [count/ktable.total for count in ktable.counts]
                kmer_freqs.append(ktable.counts)

    return kmer_freqs

def chunk_sequence(sequence, min_size, max_size):
    '''
    Cut sequences longer than 5kb into 5kb chunks and exclude trailing sequences
    if shorter than user specified min_length
    '''

    split_seqs = []
    while True:
        chunk = sequence.read(max_size)
        if len(chunk) >= min_size:
            split_seqs.append(chunk)
        else:
            break

    return split_seqs

In [3]:
contig_labels = np.load('contig_labels.npy')

In [4]:
pentamer_list = []
new_labels = []

max_size = 5000
min_size = 1000

for index, record in enumerate(SeqIO.parse('/home/jacob/Documents/Corals/Maxbin_Simulated_80x_metagenome.scaffold.fasta', "fasta")):
    s = StringIO(str(record.seq))
    split_seqs = []
    kmer_freqs = []

    #split sequence into 5kb max_size chunks
    split_seqs = chunk_sequence(s, min_size, max_size)

    #Calculate kmer frequences for each chunk
    kmer_freqs = calc_kmer_freqs(split_seqs, 5)

    pentamer_list.append(kmer_freqs)
pentamer_array = np.array(pentamer_list)

## So we have a nested list. At each level:
    contig  ---> split1 ---> (1024,) ndarray

            ---> ...

            ---> splitm ---> (1024,) ndarray

In [39]:
length = 0
for contig in pentamer_list:
    for i in range(len(contig)):
        length += 1
        
print(length)

10852


In [56]:
flatter_list = []
new_labels = []
hmm = 0
for index, contig in enumerate(pentamer_list):
    for i, kmer_freq in enumerate(contig):
        hmm += 1
        #Append correct number of corresponding labels to y
        new_labels.append(contig_labels[index])
        flatter_list.append(kmer_freq)

In [57]:
print(len(flatter_list))
print(len(new_labels))

10852
10852


In [58]:
np.save('pentamer_fixed_labels.npy', np.array(new_labels))
np.save('pentamer_fixed_data.npy', np.array(flatter_list))

In [59]:
def perform_bullshit(kmer_length, prefix):
    kmer_list = []

    max_size = 5000
    min_size = 1000

    for index, record in enumerate(SeqIO.parse('/home/jacob/Documents/Corals/Maxbin_Simulated_80x_metagenome.scaffold.fasta', "fasta")):
        s = StringIO(str(record.seq))
        split_seqs = []
        kmer_freqs = []

        #split sequence into 5kb max_size chunks
        split_seqs = chunk_sequence(s, min_size, max_size)

        #Calculate kmer frequences for each chunk
        kmer_freqs = calc_kmer_freqs(split_seqs, kmer_length)

        kmer_list.append(kmer_freqs)
    flatter_list = []
    new_labels = []
    hmm = 0
    for index, contig in enumerate(kmer_list):
        for i, kmer_freq in enumerate(contig):
            hmm += 1
            #Append correct number of corresponding labels to y
            new_labels.append(contig_labels[index])
            flatter_list.append(kmer_freq)
    np.save(prefix+'_fixed_labels.npy', np.array(new_labels))
    np.save(prefix+'_fixed_data.npy', np.array(flatter_list))
    return

In [ ]:
perform_bullshit(4, 'tetramer')
print('bullshit performed: tetramer')
perform_bullshit(6, 'sextamer')
print('bullshit performed: sextamer')
perform_bullshit(7, 'septamer')
print('bullshit performed: septamer')
perform_bullshit(8, 'octamer')
print('bullshit performed: octamer')

bullshit performed: tetramer
bullshit performed: sextamer
bullshit performed: septamer
